In [1]:
import findspark
findspark.init() 
from pyspark.sql import SparkSession
import os


MAX_MEMORY = "8g"
spark = SparkSession.builder \
                    .master("local") \
                    .appName('va_repositorio')\
                    .config("spark.executor.memory", MAX_MEMORY) \
                    .config("spark.driver.memory", MAX_MEMORY) \
                    .config("spark.jars", "E:\\va\\driverdb\\postgresql-42.2.5.jre6") \
                    .getOrCreate()

In [2]:
tabla="proyecto.view_clase_va"
df  = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://localhost:5432/va") \
    .option("dbtable", tabla) \
    .option("user", "postgres") \
    .option("password", "david") \
    .option("driver", "org.postgresql.Driver") \
    .load()
df.printSchema()

root
 |-- periodo: string (nullable = true)
 |-- estu_mcpio_reside: string (nullable = true)
 |-- estu_genero: string (nullable = true)
 |-- estu_estadocivil: string (nullable = true)
 |-- estu_etnia: string (nullable = true)
 |-- estu_limita_motriz: string (nullable = true)
 |-- estu_limita_invidente: string (nullable = true)
 |-- estu_limita_sordoconinterprete: string (nullable = true)
 |-- estu_limita_sordosininterprete: string (nullable = true)
 |-- estu_limita_sordoceguera: string (nullable = true)
 |-- estu_porcentajecreditosaprob: string (nullable = true)
 |-- estu_valormatriculauniversidad: string (nullable = true)
 |-- estu_pagomatriculabeca: string (nullable = true)
 |-- estu_pagomatriculacredito: string (nullable = true)
 |-- estu_pagomatriculapadres: string (nullable = true)
 |-- estu_pagomatriculapropio: string (nullable = true)
 |-- estu_tomo_cursopreparacion: string (nullable = true)
 |-- estu_cursodocentesies: string (nullable = true)
 |-- estu_cursoiesapoyoexterno: str

In [3]:
#CONVERTIR EL DF A TO PANDAS
df_pan=df.toPandas().drop(columns=['estu_consecutivo','estu_nse_ies','estu_limita_motriz',
                                   'estu_limita_invidente','estu_limita_sordoconinterprete','estu_limita_sordosininterprete',
                                  'estu_limita_sordoceguera','inst_origen','estu_privado_libertad'])

In [4]:
#castear tipo object string
df_pan = df_pan.astype(str)

In [6]:
df_pan.dtypes

periodo                               object
estu_mcpio_reside                     object
estu_genero                           object
estu_estadocivil                      object
estu_etnia                            object
                                       ...  
clase_va_razonamiento_cuantitativo    object
clase_va_lectura_critica              object
clase_va_competencias_ciudadanas      object
clase_va_ingles                       object
anio_preparacion                      object
Length: 61, dtype: object

In [7]:
#CONVERTIR VARIABLES A DUMMIES 
import pandas as pd
df_va_dummies= pd.get_dummies(df_pan,drop_first=True)
df_va_dummies.drop(columns=['fami_tienehornomicroogas_no_aplica',
                            'fami_tienemotocicleta_no_aplica',
                            'estu_nse_individual_no_aplica',
                            'estu_horassemanatrabaja_no_aplica',
'fami_tieneconsolavideojuegos_no_aplica',
'estu_pagomatriculabeca_no_aplica',
'estu_pagomatriculacredito_no_aplica',
'estu_pagomatriculapropio_no_aplica',
'estu_tiporemuneracion_no_aplica',
'fami_tieneinternet_no_aplica',
'estu_pagomatriculapadres_no_aplica',
'estu_etnia_no_aplica',
'fami_hogaractual_no_aplica',
'fami_cabezafamilia_no_aplica',
'estu_valormatriculauniversidad_no_aplica',
'fami_educacionmadre_no_aplica',
'fami_tienecomputador_no_aplica',
'fami_educacionpadre_no_aplica',
'fami_tienelavadora_no_aplica',
'fami_tieneautomovil_no_aplica',
'fami_tiene_serviciotv_no_aplica',
'fami_ocupacionpadre_no_aplica',
'fami_ocupacionmadre_no_aplica',
'estu_estadocivil_no_aplica',
'estu_simulacrotipoicfes_no_aplica',
'estu_actividadrefuerzoareas_no_aplica',
'estu_actividadrefuerzogeneric_no_aplica',
'estu_tomo_cursopreparacion_no_aplica',
'estu_trabaja_actualmente_no_aplica',
'fami_pisos_hogar_no_aplica',
'fami_tiene_microondas_no_aplica',
'fami_tiene_horno_no_aplica',
'fami_tiene_dvd_no_aplica',
'fami_telefono_no_aplica',
'fami_ingreso_fmiliar_mensual_no_aplica',
'fami_tiene_nevera_no_aplica',
'fami_tiene_celular_no_aplica'],inplace=True)

In [8]:
display(df_va_dummies.isnull().sum())

periodo_20123         0
periodo_20131         0
periodo_20133         0
periodo_20142         0
periodo_20143         0
                     ..
anio_preparacion_5    0
anio_preparacion_6    0
anio_preparacion_7    0
anio_preparacion_8    0
anio_preparacion_9    0
Length: 399, dtype: int64

In [9]:
df_va_dummies.head(2)

,periodo_20123,periodo_20131,periodo_20133,periodo_20142,periodo_20143,periodo_20152,periodo_20153,periodo_20162,periodo_20163,periodo_20172,...,anio_preparacion_11,anio_preparacion_12,anio_preparacion_2,anio_preparacion_3,anio_preparacion_4,anio_preparacion_5,anio_preparacion_6,anio_preparacion_7,anio_preparacion_8,anio_preparacion_9
0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0


In [8]:
#df_va_dummies.to_excel(".\\revisar.xlsx")

In [11]:
import mlxtend
from mlxtend.frequent_patterns import apriori
itemset_freq=apriori(df_va_dummies, min_support=0.1, use_colnames=True)

In [12]:
itemset_freq.tail()

,support,itemsets
157711,0.104925,"(estu_cursoiesapoyoexterno_0, clase_va_razonam..."
157712,0.107456,"(estu_cursoiesapoyoexterno_0, estu_valormatric..."
157713,0.114074,"(estu_cursoiesapoyoexterno_0, clase_va_razonam..."
157714,0.101226,"(estu_cursoiesapoyoexterno_0, estu_valormatric..."
157715,0.102978,"(estu_cursoiesapoyoexterno_0, clase_va_razonam..."


In [13]:
from mlxtend.frequent_patterns import association_rules
reglas=association_rules(itemset_freq, metric="confidence", min_threshold=0.7)

In [14]:
reglas["antecedents"] = reglas["antecedents"].apply(lambda x: ', '.join(list(x))).astype("unicode")
reglas["consequents"] = reglas["consequents"].apply(lambda x: ', '.join(list(x))).astype("unicode")

In [13]:
#reglas = reglas.astype(str)
reglas.head(5)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(2),(57),0.126338,0.824216,0.106482,0.842835,1.022590,0.002352,1.118466
1,(2),(91),0.126338,0.798131,0.120888,0.956857,1.198871,0.020053,4.679030
2,(2),(108),0.126338,0.887872,0.113685,0.899846,1.013486,0.001513,1.119552
3,(2),(112),0.126338,0.792486,0.105509,0.835131,1.053812,0.005388,1.258660
4,(2),(115),0.126338,0.481215,0.123029,0.973806,2.023641,0.062233,19.805391


In [14]:
reglas.count()

antecedents           6021962
consequents           6021962
antecedent support    6021962
consequent support    6021962
support               6021962
confidence            6021962
lift                  6021962
leverage              6021962
conviction            6021962
dtype: int64

In [15]:
#reglas = reglas.astype(str)
#reglas = reglas.write.csv("C:\Spark")
reglas.to_csv(".\\reglas.csv", encoding='utf-8', index=False)

In [16]:
reglas.dtypes

antecedents            object
consequents            object
antecedent support    float64
consequent support    float64
support               float64
confidence            float64
lift                  float64
leverage              float64
conviction            float64
dtype: object